# Build explain using weights

In [1]:
import yaml
from collections import defaultdict

In [2]:
import sys
sys.path.append('../../src/1.36/')

%load_ext autoreload
%autoreload 2

from model import Idea, Policy, Build
from loader_tools import load_ideas, load_policies
from build_tools import expand_build, BuildConst, get_total_effect

In [3]:
BUILD_CONFIG_PATH = '../../data/external/build_configs/imperial_spacemarines.yaml'

with open(BUILD_CONFIG_PATH, 'r') as f:
    build_config = yaml.safe_load(f)


HIGHLIGHTS: list[str] = build_config['highlights']

weights = build_config['weights']
COUNTRY_WEIGHTS: dict[str, float] = weights['country']
MILITARY_WEIGHTS: dict[str, float] = weights['military']
EXTRA_WEIGHTS: dict[str, float] = weights['ideas']

LIMITS = defaultdict(lambda: 100)
if 'limits' in build_config:
    LIMITS.update(build_config['limits'])

In [4]:
IDEAS: dict[str, Idea] = load_ideas([
    '../../data/processed/gypsy-1-36-1/00_basic_ideas.yaml',
    '../../data/processed/gypsy-1-36-1/00_flogi_ideas.yaml',
])

POLICIES: dict[str, Policy] = load_policies([
    '../../data/processed/gypsy-1-36-1/Idea_Variation_policies.yaml'
])

In [5]:
BUILD_IDEAS = [ 'strongman0', 'weapon_quality0', 'spy_ideas']
# BUILD_IDEAS = ['economic_ideas', 'standing_army0', 'militarism0', 'spy_ideas', 'strongman0', 'weapon_quality0', 'society0', 'imperialism0', 'quality0']

CONST = BuildConst(
    ideas = IDEAS,
    policies = POLICIES,
    country_weights = COUNTRY_WEIGHTS,
    military_weights = MILITARY_WEIGHTS,
    idea_weights = EXTRA_WEIGHTS,
    limits = LIMITS,
)


## Build

In [6]:
build = Build()
for idea_name in BUILD_IDEAS:
    build = expand_build(build, idea_name, CONST)
print(build)

Build( 367.75 - ideas: {'strongman0', 'spy_ideas', 'weapon_quality0'}


### Total effect

In [7]:
total_effect = get_total_effect(build, POLICIES)

In [8]:
for highlight in HIGHLIGHTS:
    print(f'{highlight}: {total_effect.get(highlight)}') 


development_cost: None
development_cost_in_primary_culture: None
discipline: 0.085
fire_damage: None
fire_damage_received: None
shock_damage: None
shock_damage_received: None
land_morale: None
infantry_power: 0.1
cavalry_power: 0.1
artillery_power: 0.1


In [9]:
for effect, value in total_effect.items():
    if effect not in HIGHLIGHTS:
        print(f'{effect}: {value}')

global_unrest: 4
global_tax_modifier: 0.1
production_efficiency: 0.175
defensiveness: 0.1
global_manpower_modifier: 0.2
siege_ability: 0.35
war_exhaustion: 0.05
manpower_recovery_speed: 0.25
idea_cost: 0.2
technology_cost: 0.05
global_trade_goods_size_modifier: 0.175
merchants: 1
reduced_liberty_desire: 10
num_of_pronoiars: 1
trade_efficiency: 0.15
yearly_corruption: 0.1
advisor_cost: 0.2
rebel_support_efficiency: 1
can_fabricate_for_vassals: 1
spy_offence: 0.75
embargo_efficiency: 0.33
privateer_efficiency: 0.33
diplomats: 1
discovered_relations_impact: 1
global_spy_defence: 0.5


### Idea effect highlights

In [10]:
for idea_name in build.idea_set:
    idea = IDEAS[idea_name]
    print(f'---{idea_name}---')
    for highlight in HIGHLIGHTS:
        if highlight in idea.effect:
            highlight_effect = idea.effect[highlight]
            highlight_weight = COUNTRY_WEIGHTS.get(highlight, 0) + MILITARY_WEIGHTS.get(highlight, 0)
            print(f'{highlight} - {highlight_effect} --> {highlight_weight * highlight_effect}')

---strongman0---
---spy_ideas---
discipline - 0.035 --> 70.0
---weapon_quality0---
discipline - 0.05 --> 100.0
infantry_power - 0.1 --> 80.0
cavalry_power - 0.1 --> 0.0
artillery_power - 0.1 --> 80.0


### Other idea effects

In [11]:
for idea_name in build.idea_set:
    idea = IDEAS[idea_name]
    print(f'---{idea_name}---')
    for effect, value in idea.effect.items():
        effect_weight = COUNTRY_WEIGHTS.get(effect, 0) + MILITARY_WEIGHTS.get(effect, 0)
        if effect not in HIGHLIGHTS:
            print(f'{effect} - {value} --> {effect_weight * value}')

---strongman0---
global_unrest - 2 --> 0
global_tax_modifier - 0.1 --> 0.5
production_efficiency - 0.1 --> 0.5
defensiveness - 0.1 --> 1.0
global_manpower_modifier - 0.2 --> 10.0
siege_ability - 0.15 --> 0.75
war_exhaustion - 0.05 --> 0.0
manpower_recovery_speed - 0.25 --> 5.0
idea_cost - 0.05 --> 3.0
technology_cost - 0.05 --> 3.0
global_trade_goods_size_modifier - 0.1 --> 0.5
---spy_ideas---
reduced_liberty_desire - 10 --> 0
num_of_pronoiars - 1 --> 0
trade_efficiency - 0.15 --> 0.75
yearly_corruption - 0.1 --> 0.0
advisor_cost - 0.2 --> 2.0
rebel_support_efficiency - 1 --> 0
can_fabricate_for_vassals - 1 --> 0
spy_offence - 0.75 --> 0.0
embargo_efficiency - 0.33 --> 0.0
privateer_efficiency - 0.33 --> 0.0
diplomats - 1 --> 0
discovered_relations_impact - 1 --> 0
global_spy_defence - 0.5 --> 0.0
global_unrest - 2 --> 0
---weapon_quality0---
global_trade_goods_size_modifier - 0.075 --> 0.375
merchants - 1 --> 0
siege_ability - 0.1 --> 0.5
production_efficiency - 0.075 --> 0.375


### Policies effect

In [12]:
POLICY_SLOT_INDEX = {0: ('ADM', 4), 1: ('DIP', 4), 2: ('MIL', 4)}

#### War policies

In [13]:
for i in range(3):
    policy_type = POLICY_SLOT_INDEX[i][0]
    policy_slots = POLICY_SLOT_INDEX[i][1]
    print(f'---{policy_type} ({policy_slots} slots)---')
    for j in range(min(policy_slots, len(build.war_policies[i]))):
        policy_score, policy_name = build.war_policies[i][j]
        policy = POLICIES[policy_name]
        print(f'{policy_name}:')
        for effect, value in policy.effect.items():
                effect_weight = COUNTRY_WEIGHTS.get(effect, 0) + MILITARY_WEIGHTS.get(effect, 0)
                print(f'{effect} - {value} --> {effect_weight * value}')

---ADM (4 slots)---
---DIP (4 slots)---
idea_variation_act_102:
siege_ability - 0.1 --> 0.5
---MIL (4 slots)---


#### Idea cost policies

In [14]:
for i in range(3):
    policy_type = POLICY_SLOT_INDEX[i][0]
    policy_slots = POLICY_SLOT_INDEX[i][1]
    print(f'---{policy_type} ({policy_slots} slots)---')
    for j in range(min(policy_slots, len(build.idea_policies[i]))):
        policy_score, policy_name = build.idea_policies[i][j]
        policy = POLICIES[policy_name]
        print(f'{policy_name}:')
        for effect, value in policy.effect.items():
            effect_weight = COUNTRY_WEIGHTS.get(effect, 0) + MILITARY_WEIGHTS.get(effect, 0)
            print(f'{effect} - {value} --> {effect_weight * value}')

---ADM (4 slots)---
idea_variation_act_333:
idea_cost - 0.075 --> 4.5
---DIP (4 slots)---
idea_variation_act_227:
idea_cost - 0.075 --> 4.5
---MIL (4 slots)---


#### ADM tech policies

In [15]:
for i in range(3):
    policy_type = POLICY_SLOT_INDEX[i][0]
    policy_slots = POLICY_SLOT_INDEX[i][1]
    print(f'---{policy_type} ({policy_slots} slots)---')
    for j in range(min(policy_slots, len(build.adm_tech_policies[i]))):
        policy_score, policy_name = build.adm_tech_policies[i][j]
        policy = POLICIES[policy_name]
        print(f'{policy_name}:')
        for effect, value in policy.effect.items():
            effect_weight = COUNTRY_WEIGHTS.get(effect, 0) + MILITARY_WEIGHTS.get(effect, 0)
            print(f'{effect} - {value} --> {effect_weight * value}')

---ADM (4 slots)---
---DIP (4 slots)---
---MIL (4 slots)---


#### DIP tech policies

In [16]:
for i in range(3):
    policy_type = POLICY_SLOT_INDEX[i][0]
    policy_slots = POLICY_SLOT_INDEX[i][1]
    print(f'---{policy_type} ({policy_slots} slots)---')
    for j in range(min(policy_slots, len(build.dip_tech_policies[i]))):
        policy_score, policy_name = build.dip_tech_policies[i][j]
        policy = POLICIES[policy_name]
        print(f'{policy_name}:')
        for effect, value in policy.effect.items():
            effect_weight = COUNTRY_WEIGHTS.get(effect, 0) + MILITARY_WEIGHTS.get(effect, 0)
            print(f'{effect} - {value} --> {effect_weight * value}')

---ADM (4 slots)---
---DIP (4 slots)---
---MIL (4 slots)---


#### MIL tech policies

In [17]:
for i in range(3):
    policy_type = POLICY_SLOT_INDEX[i][0]
    policy_slots = POLICY_SLOT_INDEX[i][1]
    print(f'---{policy_type} ({policy_slots} slots)---')
    for j in range(min(policy_slots, len(build.mil_tech_policies[i]))):
        policy_score, policy_name = build.mil_tech_policies[i][j]
        policy = POLICIES[policy_name]
        print(f'{policy_name}:')
        for effect, value in policy.effect.items():
            effect_weight = COUNTRY_WEIGHTS.get(effect, 0) + MILITARY_WEIGHTS.get(effect, 0)
            print(f'{effect} - {value} --> {effect_weight * value}')

---ADM (4 slots)---
---DIP (4 slots)---
---MIL (4 slots)---
